In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from src.data.loading import load_paths_and_graph

In [2]:
paths_and_graph = load_paths_and_graph()

loading raw data from tsv files...
formatting data...
extracting additional information from source code...


In [3]:
paths_and_graph["shortest-path-distance-matrix"].shape

(4604, 4604)

In [4]:
paths_and_graph["paths_finished"]

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NULL
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NULL
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NULL
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3
...,...,...,...,...,...
51313,15a13a1d66ef5456,1349231015,66,Yagan;Ancient_Egypt;Civilization,NULL
51314,2ef7ac844cefda58,1300254138,165,Yagan;Folklore;Brothers_Grimm;<;19th_century;C...,3
51315,12863abb7887f890,1385095372,228,Yagan;Australia;England;France;United_States;T...,NULL
51316,19f8284371753362,1298792567,56,Yarralumla%2C_Australian_Capital_Territory;Aus...,1


In [5]:
pd.DataFrame(
    data={
        "key": list(paths_and_graph.keys()),
        "shape": list(map(lambda d: d.shape, paths_and_graph.values()))
    }
)

,key,shape
0,shortest-path-distance-matrix,"(4604, 4604)"
1,paths_finished,"(51318, 5)"
2,articles,"(4604, 4)"
3,paths_unfinished,"(24875, 6)"
4,links,"(119882, 4)"
5,categories,"(5204, 3)"


In [6]:
paths_and_graph["links"]

,linkSource,linkTarget,linkSource_decoded,linkTarget_decoded
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Bede,Áedán_mac_Gabráin,Bede
1,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Columba,Áedán_mac_Gabráin,Columba
2,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,D%C3%A1l_Riata,Áedán_mac_Gabráin,Dál_Riata
3,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Great_Britain,Áedán_mac_Gabráin,Great_Britain
4,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Ireland,Áedán_mac_Gabráin,Ireland
...,...,...,...,...
119877,Zulu,South_Africa,Zulu,South_Africa
119878,Zulu,Swaziland,Zulu,Swaziland
119879,Zulu,United_Kingdom,Zulu,United_Kingdom
119880,Zulu,Zambia,Zulu,Zambia
